## Read files like Be8_SIII_new.txt

**Naming convention: YYMMDD_Be8**

To use this notebook to generate input for vqe:
1. Provides source file name labeled(#filename)
2. Parameters are to be output in the format of a list 
```
the list = [num_orbitals,num_spin_orbitals,num_particles,num_spatial_orbitals]
```




### DATA ID
# 210_Be9




In [1]:
data_ID = "210_Be9"

In [2]:
from utils import *
## Computation config ## 
pcname = "Hpc" #or "Hlp" or Ypc"
quan_algo = "VQE"    ## (string)(VQE or adaptVQE)
## Source File input ##

### The following doesnt require setting up, unless file naming is different
obs_onebody_csv = data_ID+"/"+data_ID+"-1B_H_input.csv"
obs_twobody_csv = data_ID+"/"+data_ID+"-2B_H_input.csv"
parameter_py = data_ID+"/"+"parameter.txt"

def pathfilename_gen(pcname_:"string", input_dir_:"string")->"string , string , dict":
    ## Setting up the path (now is directory where compute.py is ran)
    current_path = os.getcwd()                     # Current path(which is the Compute dir)
    try:
        abs_main                                   # Check if the abs_main was defined before
    except:
        abs_main = os.path.dirname(current_path)   # Main directory containing the compute dir
    os.chdir(abs_main)                             # Change dir to main dir(so called absolute main directory)

    abspath_data_dir = os.path.join(abs_main,"Data")
    abspath_result_dir = os.path.join(abs_main,"Result")
    
    #### Extract index for next dir and filename
    rel_path_result = os.path.relpath("Result",abs_main)
    rel_PATH_result = (Path(rel_path_result))
    subresult_dir_list = [str(x) for x in rel_PATH_result.iterdir() if x.is_dir()]
    if len(subresult_dir_list) == 0:
        new_index = 1
    else:
        latest_result_dir = (max(subresult_dir_list,key=extract_number))
        new_index = extract_number(latest_result_dir)[0]+1
    
    #### Create a new directory to keep new results
    ##### Extract neucleus name
    input_dir_dum = re.split('-|_|\\.', input_dir_)
    nucleus_name = input_dir_dum[1]
    output_id = pcname_ + "_" + "{:03d}".format(new_index) + "_"
    subresult_dir_name = os.path.join(rel_path_result,output_id+nucleus_name)
    os.mkdir(subresult_dir_name)
    subresult_dir = os.path.join(subresult_dir_name,output_id+nucleus_name)
    ### Using a dictionary to store the file names to be used
    pathfilename = {}
    #### Input filenames or path
    pathfilename["source_text"] = os.path.join(abspath_data_dir, input_dir_)
    #### Output # only ready to be input as fermionic op is considered not source
    pathfilename["output_1B-source_csv"] =  subresult_dir + "-1B-source.csv"
    pathfilename["output_2B-source_csv"] =  subresult_dir + "-2B-source.csv"
    pathfilename["output_1B-H_input_csv"] = subresult_dir + "-1B-H_input.csv"
    pathfilename["output_2B-H_input_csv"] = subresult_dir + "-2B-H_input.csv"
    pathfilename["abstract_result"] = subresult_dir + "-vqe_abst.txt"
    pathfilename["full_result"] = subresult_dir + "-vqe_full.txt"
    return abs_main, nucleus_name, pathfilename

# abs_main, nucleus, pathfilename = pathfilename_gen(pcname,input_dir)
# pathfilename

In [3]:
text_two = []
with open("source\Be9_SIII_0508\Vpair_Vdelta_800_Be9_EFA_lower.txt", 'rt') as myfile:    #filename
# with open("source/Be8_SIII_new/Vpair_Vdelta_800.txt", 'rt') as myfile:    #filename
    for line in myfile:
        line = line.rstrip('\n')
        text_two.append(line)

# Parameters

In [4]:
num_orbitals = (6,6)
num_particles = (5,4)
num_spin_orbitals = 12
num_spatial_orbitals = int(num_spin_orbitals/2)

# Onebody source setup (copy nia)

In [5]:
import pandas as pd
index_list = list(range(0,12,1))

# include all 20 orbitals 
# energy = {"index": index_list, 
#           "epsilon":[-25.399,-25.399,-12.640,-12.640,-9.353,-9.353,-2.927,-2.927,-1.822,-1.822,
#                     -27.370,-27.370, -13.513,-13.513,-11.604,-11.604,-4.744,-4.744,-2.024, -2.024,]}
# only 12 orbitals (since the interaction is upto only orbital 6)
energy = {"index": index_list, 
          "epsilon":[-25.399,-25.399,-12.640,-12.640,-9.353,-9.353,
                    -27.370,-27.370, -13.513,-13.513,-11.604,-11.604,]}

obs_onebody_df_source = pd.DataFrame(energy)
obs_onebody_df_source

,index,epsilon
0,0,-25.399
1,1,-25.399
2,2,-12.640
3,3,-12.640
4,4,-9.353
5,5,-9.353
6,6,-27.370
7,7,-27.370
8,8,-13.513
9,9,-13.513


# Twobody Matrix Terms source

In [6]:
## the line being popped, is the line with labels, the list is preprocessed for df construct 
import re
pattern = re.compile("isospin")
if pattern.search(text_two[0]) is not None:
    text_two.pop(0)

pairs = [[1,2],[3,4],[5,6],[7,8],[9,10]]
obs_twobody_df_source = pd.DataFrame(columns =["q_i1", "q_i2", "q_f1", "q_f2", "V_ffii"])
for counter, line in enumerate(text_two): 
    splitted = line.split()
    if int(splitted[0]) == 1:
        for x in pairs:
            if int(splitted[1]) in x:
                init = x
            if int(splitted[2]) in x:
                fina = x
        init = (init[0]-1,init[1]-1) # minus 1 because the index in python starts with 0
        fina = (fina[0]-1,fina[1]-1)
        V_pair = float(splitted[3])
    if int(splitted[0]) ==2:
        for x in pairs:
            if int(splitted[1]) in x:
                init = x
            if int(splitted[2]) in x:
                fina = x
        init = (init[0]+5,init[1]+5) # plus 5 because the index in python starts with 0; and proton'index starts after neutron's
        fina = (fina[0]+5,fina[1]+5)
        V_pair = float(splitted[3])
    V_pair = round(V_pair,6)
    obs_twobody_df_source.loc[counter] = [init[0],init[1],fina[0],fina[1],round(V_pair,5)]
    obs_twobody_df_source.loc[3*counter] = [fina[0],fina[1],init[0],init[1],round(V_pair,5)]
    
#     obs_twobody_df_source.loc[6*counter] = [init[1],init[0],fina[0],fina[1],round(V_pair,5)]
#     obs_twobody_df_source.loc[9*counter] = [init[1],init[0],fina[1],fina[0],round(V_pair,5)]
#     obs_twobody_df_source.loc[12*counter] = [init[0],init[1],fina[1],fina[0],round(V_pair,5)]
#     obs_twobody_df_source.loc[15*counter] = [fina[1],fina[0],init[0],init[1],round(V_pair,5)]
#     obs_twobody_df_source.loc[18*counter] = [fina[1],fina[0],init[1],init[0],round(V_pair,5)]
#     obs_twobody_df_source.loc[21*counter] = [fina[0],fina[1],init[1],init[0],round(V_pair,5)]
    del init, fina
obs_twobody_df_vqe = obs_twobody_df_source.copy()

In [7]:
obs_twobody_df_vqe.drop_duplicates(inplace=True)
obs_twobody_df_vqe = obs_twobody_df_vqe.drop_duplicates().reset_index()
obs_twobody_df_vqe

,index,q_i1,q_i2,q_f1,q_f2,V_ffii
0,0,0.0,1.0,0.0,1.0,-12.92433
1,1,0.0,1.0,2.0,3.0,-5.92483
2,3,2.0,3.0,2.0,3.0,-6.22180
3,2,0.0,1.0,4.0,5.0,-6.42081
4,6,6.0,7.0,6.0,7.0,-12.52288
5,9,8.0,9.0,8.0,9.0,-7.12469
6,4,2.0,3.0,4.0,5.0,-3.79370
7,5,4.0,5.0,4.0,5.0,-6.13790
8,7,6.0,7.0,8.0,9.0,-6.28347
9,8,6.0,7.0,10.0,11.0,-6.49777


In [8]:
# ## To generate twobody terms that elimiated the first 2 two hartree fock states 
# ## (basically consider a helium core, and dont allow the lowest pair to participate the pariing interaction)

# for index, row in obs_twobody_df_vqe.iterrows():
#     if (row['q_i1'] == 0) or (row['q_f1'] == 0) or (row['q_i1'] == 6) or (row['q_f1'] == 6):
#         obs_twobody_df_vqe.drop(index,inplace=True)

# obs_twobody_df_vqe

# Filter out unwanted interactions from obs_twobody_df
# Create obs_twobody_H_input_df_output (input for VQE)

In [9]:
obs_twobody_df_vqe=obs_twobody_df_vqe.sort_values(["q_i1","q_f1"])
obs_twobody_df_vqe = obs_twobody_df_vqe.reset_index()
obs_twobody_df_vqe = obs_twobody_df_vqe[["q_i1","q_i2","q_f1","q_f2","V_ffii"]]
obs_twobody_df_vqe.to_csv(obs_twobody_csv)
obs_twobody_df_vqe

PermissionError: [Errno 13] Permission denied: '210_Be9/210_Be9-2B_H_input.csv'

# Create obs_onebody_df_vqe (input for VQE)

In [ ]:
obs_onebody_df_source

In [ ]:
# fermin = (-10.853 + (-4.162))/2
# fermip = (-15.475 + (-15.459))/2

### Below consider energy differences between levels
obs_onebody_df_vqe = pd.DataFrame(columns =['q_i','q_f','epsilon'])

### init_fina_list
epsilon_list = obs_onebody_df_source.loc[:,"epsilon"]
import math
for counter, i in enumerate(epsilon_list):
    epsilon = i
#     if counter < num_orbitals[0]:
#         epsilon = math.sqrt((i - (fermin))**2)
#     if counter >= num_orbitals[0]:
#         epsilon = math.sqrt((i - (fermip))**2)
    obs_onebody_df_vqe.loc[counter] = [counter,counter, round(epsilon,6)]

obs_onebody_df_vqe.to_csv(obs_onebody_csv)
obs_onebody_df_vqe

In [ ]:
# %store promotion_list
# %store num_orbitals
# %store num_particles
# %store num_spin_orbitals
# %store obs_onebody_df_vqe
# %store obs_twobody_df
# %store abs_main
# %store pathfilename
# %store nucleus_name

In [13]:
obs_twobody_df = pd.read_csv("210_Be9/210_Be9-2B_H_input.csv")

'C:\\Users\\chwee\\OneDrive\\UTM Directories\\QISKIT and codes\\Workplace\\VQE_pair\\Data'

In [16]:
obs_twobody_df = obs_twobody_df_vqe
from qiskit_nature.second_q.operators import FermionicOp
import pandas as pd
tmp_ham = {}
two_factor = -0.5
### Two body Terms: Pairing interaction
for index, row in obs_twobody_df.iterrows():
    init_1 = int(row['q_i1']); init_2 = int(row['q_i2']);
    fina_1 = int(row['q_f1']); fina_2 = int(row['q_f2']);
    the_twostring = "+_" +str(fina_1) + " " + "+_" +str(fina_2) + " " + "-_" +str(init_1) + " " + "-_" +str(init_2)
    tmp_ham[the_twostring] = two_factor*row['V_ffii']
len(tmp_ham)

9

In [17]:
Hamiltonian = FermionicOp(tmp_ham, 
                          num_spin_orbitals=num_spin_orbitals, 
                          copy=False)


In [18]:
print(Hamiltonian)

Fermionic Operator
number spin orbitals=12, number terms=9
  6.462165 * ( +_0 +_1 -_0 -_1 )
+ 3.1109 * ( +_2 +_3 -_2 -_3 )
+ 6.26144 * ( +_6 +_7 -_6 -_7 )
+ 3.248885 * ( +_10 +_11 -_6 -_7 )
+ 3.562345 * ( +_8 +_9 -_8 -_9 )
+ 1.91609 * ( +_10 +_11 -_8 -_9 )
+ 3.248885 * ( +_6 +_7 -_10 -_11 )
+ 1.91609 * ( +_8 +_9 -_10 -_11 )
+ 3.167165 * ( +_10 +_11 -_10 -_11 )


In [19]:
Hamiltonian.is_hermitian()

True